# Installation
To get started, run the following command to install all required dependencies.

In [ ]:
#!pip install -q -r ./requirements.txt

# Import requirements
Here we import all external and local modulues.

In [ ]:
from fpldata.common import Context, is_notebook

from fpldata.prepare import prepare_game_weeks, get_next_gw_counts, prepare_teams, get_team_id_code_map, \
    prepare_fixtures, get_fixtures_id_code_map, prepare_players, get_players_id_code_map, \
    prepare_players_history_past, prepare_players_history, get_team_goal_stats_est
from fpldata.derive import get_player_teams, get_fixture_teams, get_players_history_fixtures, \
    get_team_goal_stats, get_fixture_goal_stats, get_team_fixture_strength, get_player_team_fixture_strength, \
    get_team_future_fixtures, get_players_future_fixture_team_strengths, get_player_fixture_stats, get_players_fixture_team_eps, \
    get_players_gw_team_eps, get_player_gw_next_eps
from fpldata.backtest import get_gw_points_backtest
from fpldata.s3store import S3Store
from fpldata.manager import FPLManager
from fpldata.test import FPLManagerTest
from fplpandas import FPLPandas

import os
import logging, sys
import pandas as pd
from datadict import DataDict
from typing import Dict

# Define type aliases
DF = pd.DataFrame
S = pd.Series

# Set variables
This section sets all important global variables.

In [ ]:
ROOT_DIR = os.path.dirname(__file__) if '__file__' in globals() else '.'
DATA_DICT_FILE = f'{ROOT_DIR}/data/data_dictionary.csv'

S3_BUCKET_PROD = 'fpl.177arc.net' # S3 bucket for exporting the production data sets.
S3_BUCKET_TEST = 'fpl-test.177arc.net' # S3 bucket for exporting the test data sets.

LAST_SEASON = '2019-20'
CURRENT_SEASON = '2020-21'
FIXTURE_LOOK_BACK = 20 # Limit of how many fixtures to look back for calculating rolling team stats
PLAYER_FIXTURE_LOOK_BACK = 12 # Limit of how many fixture to look back for calculating rolling player stats

# Configure logging
logging.basicConfig(level=(logging.WARN if is_notebook() else logging.INFO))

pd.set_option('display.max_columns', 100)

ENV = os.environ.get('ENV')
ENV = 'Comp-Test'
# Wrapper for access to variables and functionality that is specifiic to the mode that we are running in.
if ENV == 'Prod':
    fpl = FPLManager(LAST_SEASON, CURRENT_SEASON, S3_BUCKET_PROD, FIXTURE_LOOK_BACK, PLAYER_FIXTURE_LOOK_BACK)
    logging.warning(f'Running in live mode and will publish to {S3_BUCKET_PROD} ...')
elif ENV == 'Comp-Test':
    fpl = FPLManagerTest(f'{ROOT_DIR}/tests/comp')
    logging.info(f'Running in test mode ...')
else:
    fpl = FPLManager(LAST_SEASON, CURRENT_SEASON, S3_BUCKET_TEST, FIXTURE_LOOK_BACK, PLAYER_FIXTURE_LOOK_BACK)
    logging.warning(f'Running in live mode and will publish to {S3_BUCKET_TEST} ...')

# Load data dictionary
This section loads the data dictionary. The data dictionary contains default ordering of fields, for each field a description, default format and mapping of API field names to more readable ones. It is used to show data in a more user-friendly way.

In [ ]:
ctx = fpl.create_context()
ctx.dd = DataDict(data_dict_file=DATA_DICT_FILE)

# Load game week data
The data frame contains one row for each game week for the current season.

In [ ]:
logging.info('Loading game week data ...')

gws = fpl.get_game_weeks().pipe(prepare_game_weeks, ctx)
ctx.total_gws = gws.shape[0]
ctx.next_gw = gws[lambda df: df['Is Next GW?']].index.values[0]
ctx.def_next_gws = 'Next 8 GWs'
ctx.next_gw_counts = get_next_gw_counts(ctx)

fpl.assert_context(ctx)

# Load team data
This section loads the team data and stats from the following endpoint: https://fantasy.premierleague.com/api/bootstrap-static/ and returns it as a panda data frame.

In [ ]:
logging.info('Loading team data ...')

# Get current team data. The resultnig data frame contains one row for each team playing in the current season.
teams = fpl.get_teams().pipe(prepare_teams, ctx)
team_id_code_map = teams.pipe(get_team_id_code_map)

In [ ]:
# Get last season's team data
teams_last_season = fpl.get_teams_last_season().pipe(prepare_teams, ctx)
teams_last_season_id_code_map = teams_last_season.pipe(get_team_id_code_map)

# This data frame contains one row for each team playing in the current season and the past season.
teams_ext = pd.concat([teams, teams_last_season[~teams_last_season.index.isin(teams.index)]])

# Load fixture data
This section loads the fixture data and stats from the following endpoint: https://fantasy.premierleague.com/api/fixtures/ and returns it as a panda data frame.

In [ ]:
logging.info('Loading fixture data ...')

# Get current fixture data. The resulting data frame contains one row for each fixture of the current season, both past and future ones.
fixtures = fpl.get_fixtures().pipe(prepare_fixtures, team_id_code_map, ctx).assign(**{'Season': ctx.current_season})
fixtures_id_code_map = fixtures.pipe(get_fixtures_id_code_map)

In [ ]:
# Get last season's fixture data
fixtures_last_season = (fpl.get_fixtures_last_season()
                        .pipe(prepare_fixtures, teams_last_season_id_code_map, ctx)).assign(**{'Season': ctx.last_season})
fixtures_last_season_id_code_map = fixtures_last_season.pipe(get_fixtures_id_code_map)

# This data frame contains one row for each fixture of the the last and the current season, both past and future ones.
fixtures_ext = pd.concat([fixtures, fixtures_last_season])

# Load player data
This section loads the player data and stats from the following FPL API endpoint: https://fantasy.premierleague.com/api/bootstrap-static/ and returns it as a panda data frame. **This can take a few seconds** because for each player the full history for the current season is downloaded.

In [ ]:
logging.info('Loading player data ...')

# Get current player data
players_raw = fpl.get_players()
players = (players_raw[0]
           .pipe(prepare_players, ctx)
           .assign(**{'Season': ctx.current_season}))
players_id_code_map = players.pipe(get_players_id_code_map)


# This data frame contains one row for every past season played in the premier league for every player in the current season.
#players_history_past = players_raw[1].pipe(prepare_players_history_past, players_id_code_map, ctx)

# This data frame contains one row for every completed fixture in the current season for every player.
players_history = (players_raw[2]
                   .pipe(prepare_players_history, players_id_code_map, fixtures_id_code_map, ctx)
                   .assign(**{'Season': ctx.current_season}))

In [ ]:
# Get last season's player data
players_last_season_raw = fpl.get_players_last_season()
players_last_season =  (players_last_season_raw[0]
                        .pipe(prepare_players, ctx).assign(**{'Season': ctx.last_season}))
players_last_season_id_code_map = players_last_season.pipe(get_players_id_code_map)

players_history_last_season = (players_last_season_raw[2]
            .pipe(prepare_players_history, players_last_season_id_code_map, fixtures_last_season_id_code_map, ctx)
            .assign(**{'Season': ctx.last_season}))

# This data frame contains one row for every player in the last and the current season.
players_ext = pd.concat([players, players_last_season[~players_last_season.index.isin(players.index)]])

# This data frame contains one row for every completed fixuture in the last and the current season.
players_history_ext = pd.concat([players_history, players_history_last_season])

# Create derived data
This section creates new dataset by combining the previously loaded ones.

In [ ]:
logging.info('Creating derived data sets ...')

## Players with team info

In [ ]:
player_teams = players.pipe(get_player_teams, teams, ctx)
player_teams_ext = players.pipe(get_player_teams, teams_ext, ctx)

## Fixtures with team info

In [ ]:
fixture_teams_ext = fixtures_ext.pipe(get_fixture_teams, teams_ext, ctx)

## Player derived fields and metrics
The section below derives a few useful player attributes but most importantly, it calculates the total points earned by a player devided by his current cost. This is can be an indicator for whether the player is undervalued or overpriced.

In [ ]:
players_history_fixtures_ext = players_history_ext.pipe(get_players_history_fixtures, fixtures_ext, player_teams_ext, ctx)

## Team metrics

In [ ]:
team_goal_stats_est = get_team_goal_stats_est(fpl.get_last_season_stats_est())
team_goal_stats_ext = fixture_teams_ext.pipe(get_team_goal_stats, teams_ext, team_goal_stats_est, ctx)
fpl.assert_team_goal_stats_ext(team_goal_stats_ext)

## Fixture stats
In order to calculate the relative strengths of each player for each fixture, we first need to transfer the team score stats to the fixtures.

In [ ]:
fixture_goal_stats_ext = fixture_teams_ext.pipe(get_fixture_goal_stats, team_goal_stats_ext)

## Calculate relative fixture strengths
The fundamental idea is that the best evidence for a player's ability to generate points is to look over
a sliding window of past fixtures while taking into account the difficulty of the opposing team.

The expected points for each game week for each player are calculated by taking the average points earned by  
each player for every event type (e.g. goals scored, goals conceded, clean sheets, yellow cards, etc.) separately
over a sliding window of past fixtures (currently 12). These averages are adjusted based on the relative strength
of the opposing team compared to the relative strength of the opposing teams that the player has played so far.

So first we need to calculate the relative fixtures strengths for each team based on the goals scored/conceded at home and away. The relative strength is a factor around 1 and is used in the expected point prediction below to adjust the predicted points based on the relative strengths of the upcoming fixtures. A value above 1 indicates that the player's team is relatively stronger and a value below 1 indicates that the team is relatively weaker.

Four different strength factors are calculated:
- `Rel Att Fixture Strength Home` and `Rel Att Fixture Strength Away`: Relative attacking strength of the team for the fixture for scaling the expected points of mid fielders and forwards both at home and away
- `Rel Def Fixture Strength Home` and `Rel Def Fixture Strength Away`: Relative defensive strength of the team for the fixture for scaling the expected point of the defenders and goal keepers both at home and away

### Relative attacking strengths
The relative attacking strength of the opposing team for a specific fixture is the average number of goals conceded by teams the team played against over the same sliding window by divided the number of goals conceded over a sliding window of past fixtures (currently 38) by the opposing team. This calculated separately for both home and away games.

$$s_{A,H/A} = \frac{\frac{\sum_{i=1}^{F_{T}} \frac{\sum_{i=1}^{F_{O}} g_{O,C,A/H,i}}{F}}{F}}{\frac{\sum_{i=1}^{F_{O}} g_{O,C,A/H,i}}{F}}$$

where

* $g_{O,C,A/H}$: Goals conceded by the opposing team at away/at home.
* $F$: Number of fixtures in the sliding window.
* $F_{T}$: Fixtures played by this team in the sliding window.
* $F_{O}$: Fixtures played by opposing team in the sliding window.

### Relative defensive strengths
The relative defensive strength of the opposing team for a specific fixture is simply the number of goals scored over a sliding window of past fixtures (currently 38) by the opposing team divided by the average number of goals scored by teams the team played against over the same sliding window. This calculated separately for both home and away games.

$$s_{D,H/A} = \frac{\frac{\sum_{i=1}^{F_{O}} g_{O,S,A/H,i}}{F}}{\frac{\sum_{i=1}^{F_{T}} \frac{\sum_{i=1}^{F_{O}} g_{O,S,A/H,i}}{F}}{F}}$$

where

* $g_{O,S,A/H}$: Goals scored by the opposing team at away/at home.
* $F$: Number of fixtures in the sliding window.
* $F_{T}$: Fixtures played by this team in the sliding window.
* $F_{O}$: Fixtures played by opposing team in the sliding window.


### Limitations
This approach assumes that the points awarded to a player are linear with the number of goals scored or conceded. This is clearly not the case since the formula for calculating is much more complicated. Nevertheless this is a reasonable approximation and we can improve on this at a later point.

In [ ]:
team_fixture_stats_ext = fixture_goal_stats_ext.pipe(get_team_fixture_strength, teams_ext, ctx)

## Transfer relative fixture strengths from fixtures to players
This section joins the fixture strengths data set with the player data set so that expected points can be calculated on a player basis.

In [ ]:
player_team_fixture_strength_ext = players.pipe(get_player_team_fixture_strength, team_fixture_stats_ext, players_history_ext, ctx)

## Create combined data for past and future fixtures for each player
This section concatenates two sets: one historical and one future fixture set. The reason for this is that for completed matches, we need it to consider the team that player actually played for, while for future games we can assume that the player will play for the same team than he is currently in.

In [ ]:
team_future_fixtures = team_fixture_stats_ext.pipe(get_team_future_fixtures, players_history_fixtures_ext)
players_future_fixture_team_strengths = player_teams.pipe(get_players_future_fixture_team_strengths, team_future_fixtures)
player_fixture_stats = players_history_fixtures_ext.pipe(get_player_fixture_stats, players_future_fixture_team_strengths, player_team_fixture_strength_ext)

## Calculates the expected points for different  time horizons
In this section, we first calculate the expected points for each game week and each player in two ways:
* *Simple*: The expected points for a future fixture is the average of points over a sliding window of past fixtures.
* *Based on relative strength*: Similar to the simple calculation but instead of using the total points, we calculate the average points for each event type (e.g. goals scored, goals conceded, clean sheets, yellow cards, etc.) over the same sliding window of past fixtures and scale them depending on the relative attacking/defensive strength of the future fixture as calculated in the previous section.

Note we calculate the expected points also for past fixtures for the purpose back testing.

We then calculate the cumulative expected points for the all the game weeks up to the end of the season. The expected points for each time horizon are simply the sum of expected points for each game week within the time horizon.

Calculate expected points for each player and fixture combination.

In [ ]:
players_fixture_team_eps_ext = player_fixture_stats.pipe(get_players_fixture_team_eps)

Project the fixtures to game week level to deal with game weeks when there is no fixture for a team or double fixtures.

In [ ]:
players_gw_team_eps_ext = players_fixture_team_eps_ext.pipe(get_players_gw_team_eps, player_teams)

Calculates the expected points for the different time horizons for each player.

In [ ]:
player_gw_next_eps_ext = players_gw_team_eps_ext.pipe(get_player_gw_next_eps, ctx)

## Back test the expected points

In [ ]:
gw_points_backtest = get_gw_points_backtest(players_gw_team_eps_ext, ctx)
backtest_results = dict(gw_points_backtest[['Error', 'Sq Error', 'Error Simple', 'Sq Error Simple']].mean())
print(f'Back test results for expected points:'
      +f'\nMean error based on relative strength: {backtest_results["Error"]:.2f} points'
      +f'\nMean error based on simple approach: {backtest_results["Error Simple"]:.2f} points'
      +f'\nMean square error based on relative strength: {backtest_results["Sq Error"]:.2f} points'
      +f'\nMean Square error based on simple approach: {backtest_results["Sq Error Simple"]:.2f} points')
assert backtest_results['Error'] < backtest_results['Error Simple'], 'Error of the Expected Points based on the complex formula are worse than the error of the Expected Points based on the simple calculation.'

In [ ]:
if is_notebook():
    import plotly.express as px
    px.line(gw_points_backtest, x='Season Game Week', y=['Avg Expected Points', 'Avg Expected Points Simple', 'Avg Fixture Total Points', 'Error']).show()

In [ ]:
fpl.assert_player_gw_next_eps_ext(player_gw_next_eps_ext)

# Publish data sets

In [ ]:
fpl.publish_data_sets(globals())